In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [2]:
# ⚙️ Config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"  # For testing purposes, use CPU

In [3]:
device

device(type='cuda')

In [4]:
class HorizonSeqDatasetWithContext(Dataset):
    def __init__(self, df):
        rewards = df[[f"r{i}" for i in range(1, 11)]].values
        choices = df[[f"c{i}" for i in range(1, 11)]].values
        game_lengths = df["gameLength"].values
        ucs = df["uc"].values

        choices = choices - 1  # Ensure binary [0,1]

        self.inputs, self.targets, self.contexts = [], [], []

        for r_seq, c_seq, gl, uc in zip(rewards, choices, game_lengths, ucs):
            valid_steps = int(gl)
            if valid_steps <= 1:
                continue
            x = np.stack([r_seq[:valid_steps - 1], c_seq[:valid_steps - 1]], axis=1)
            y = c_seq[1:valid_steps]
            if not (np.isnan(x).any() or np.isnan(y).any()):
                self.inputs.append(torch.tensor(x, dtype=torch.float32))
                self.targets.append(torch.tensor(y, dtype=torch.float32))
                self.contexts.append(torch.tensor([gl, uc], dtype=torch.float32))

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx], self.contexts[idx]

In [5]:
def variable_length_collate(batch):
    input_seqs = [item[0] for item in batch]
    target_seqs = [item[1] for item in batch]
    contexts = [item[2] for item in batch]
    return input_seqs, target_seqs, contexts


In [6]:

# Load and split dataset
df = pd.read_csv("data/allHorizonData_cut.csv")
dataset = HorizonSeqDatasetWithContext(df)

train_idx, val_idx = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42)
train_loader = DataLoader(Subset(dataset, train_idx), batch_size=64, shuffle=True,
                          collate_fn=variable_length_collate)

val_loader = DataLoader(Subset(dataset, val_idx), batch_size=64,
                        collate_fn=variable_length_collate)

In [7]:


class TinyGRUWithContext(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=2, context_dim=2):
        super().__init__()
        self.rnn_cell = nn.GRUCell(input_size=input_dim, hidden_size=hidden_dim)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim + context_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, input_seq, context):
        # input_seq: [1, T, 2]
        # context: [2]
        T = input_seq.size(1)
        h = torch.zeros(1, self.rnn_cell.hidden_size, device=input_seq.device)
        outputs = []

        for t in range(T):
            x_t = input_seq[:, t, :]
            h = self.rnn_cell(x_t, h)
            h_ctx = torch.cat([h, context.unsqueeze(0)], dim=1)  # [1, hidden + context]
            out = self.output_layer(h_ctx)
            outputs.append(out)

        return torch.cat(outputs, dim=0).squeeze(1)  # [T]

In [ ]:

# 🏋️ Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyGRUWithContext().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

EPOCHS = 30
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

for input_batch, target_batch, context_batch in train_loader:
    for input_seq, target_seq, context in zip(input_batch, target_batch, context_batch):
        input_seq = input_seq.unsqueeze(0).to(device)  # [1, T, 2]
        target_seq = target_seq.to(device)             # [T]
        context = context.to(device)                   # [2]

        preds = model(input_seq, context)              # ✅ pass both inputs here

        loss = criterion(preds, target_seq)
        loss.backward()
        optimizer.step()


        pred_bin = (preds > 0.5).float()
        all_preds.extend(pred_bin.cpu().numpy())
        all_labels.extend(target_seq.cpu().numpy())


    acc = accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(train_loader)
    if epoch % 5 == 0:
        print(f"Epoch {epoch}: Loss = {avg_loss:.4f}, Accuracy = {acc:.4f}")


In [ ]:


model.eval()

preds_all, labels_all = [], []
preds_H1, labels_H1 = [], []
preds_H6, labels_H6 = [], []

with torch.no_grad():
    for input_batch, target_batch, context_batch in val_loader:
        for input_seq, target_seq, context in zip(input_batch, target_batch, context_batch):
            input_seq = input_seq.unsqueeze(0).to(device)   # [1, T, 2]
            target_seq = target_seq.to(device)              # [T]
            context = context.to(device)                    # [2]

            preds = model(input_seq, context)               # [T]
            pred_bin = (preds > 0.5).float()

            preds_all.extend(pred_bin.cpu().numpy())
            labels_all.extend(target_seq.cpu().numpy())

            if context[0].item() == 5:
                preds_H1.extend(pred_bin.cpu().numpy())
                labels_H1.extend(target_seq.cpu().numpy())
            elif context[0].item() == 10:
                preds_H6.extend(pred_bin.cpu().numpy())
                labels_H6.extend(target_seq.cpu().numpy())

# 🧮 Accuracy calculations
acc_all = accuracy_score(labels_all, preds_all)
acc_H1 = accuracy_score(labels_H1, preds_H1)
acc_H6 = accuracy_score(labels_H6, preds_H6)

# 📢 Print
print(f"\nOverall Accuracy  : {acc_all:.4f}")
print(f"Horizon 1 Accuracy: {acc_H1:.4f}")
print(f"Horizon 6 Accuracy: {acc_H6:.4f}")




Overall Accuracy  : 0.5081
Horizon 1 Accuracy: 0.5061
Horizon 6 Accuracy: 0.5090
